#### Dependencias

In [ ]:
import pyodbc
import pandas as pd
from credenciales_sqlserver import *

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

#### Parámetros de la conexión

- <i style="color:yellow">Habría que comentar y descomentar la versión del driver que se encuentra instalado en el sistema donde va a correr el script o el notebook.</i>.
- <i style="color:yellow">Los datos de conexión (IP, usuario, clave, puerto) no pueden alterarse ya que corresponden a la configuración del servidor</i>

In [ ]:


connection_string = (
    # f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    # f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"DRIVER={{SQL Server}};"
    f"SERVER={SERVER},{PORT};"
    f"DATABASE={DATABASE};"
    f"UID={USERNAME};"
    f"PWD={PASSWORD}"
)


Con la siguiente línea puedes ver los controladores que tienes instalados.

<code>print(pyodbc.drivers())</code>

#### Funciones típicas

In [3]:
def get_connection():
    return pyodbc.connect(connection_string)

In [4]:
def crear_tabla():
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute("""
            IF OBJECT_ID('Empleados', 'U') IS NULL
            CREATE TABLE Empleados (
                ID INT PRIMARY KEY IDENTITY(1,1),
                Nombre NVARCHAR(100),
                Edad INT
            )
        """)
        conn.commit()
    print("✅ Tabla 'Empleados' creada o ya existente.")


In [5]:
def insertar_empleado(nombre, edad):
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute("INSERT INTO Empleados (Nombre, Edad) VALUES (?, ?)", (nombre, edad))
        conn.commit()
    print(f"✅ Insertado: {nombre}, {edad} años.")


In [6]:
def editar_empleado(id_empleado, nuevo_nombre, nueva_edad):
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute("""
            UPDATE Empleados
            SET Nombre = ?, Edad = ?
            WHERE ID = ?
        """, (nuevo_nombre, nueva_edad, id_empleado))
        conn.commit()
    print(f"✏️ Empleado con ID {id_empleado} actualizado.")


In [7]:
def eliminar_empleado(id_empleado):
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute("DELETE FROM Empleados WHERE ID = ?", (id_empleado,))
        conn.commit()
    print(f"🗑️ Empleado con ID {id_empleado} eliminado.")


In [8]:
def eliminar_tabla():
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute("DROP TABLE IF EXISTS Empleados")
        conn.commit()
    print("💥 Tabla 'Empleados' eliminada si existía.")


In [9]:
def listar_empleados():
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM Empleados")
        for row in cursor.fetchall():
            print(f"ID: {row.ID}, Nombre: {row.Nombre}, Edad: {row.Edad}")


In [13]:
def pd_from_tabla(tabla):
    with get_connection() as conn:
        query = f"SELECT * FROM {tabla}"
        df = pd.read_sql(query, conn)
    return df

#### Ejecución del código

In [34]:
crear_tabla()

✅ Tabla 'Empleados' creada o ya existente.


In [11]:
insertar_empleado("Juan Pérez", 30)
insertar_empleado("Ana Gómez", 25) 
insertar_empleado("Luis Rodríguez", 40)
insertar_empleado("María López", 28)
insertar_empleado("Carlos Sánchez", 35)

✅ Insertado: Juan Pérez, 30 años.
✅ Insertado: Ana Gómez, 25 años.
✅ Insertado: Luis Rodríguez, 40 años.
✅ Insertado: María López, 28 años.
✅ Insertado: Carlos Sánchez, 35 años.


In [12]:
listar_empleados()

ID: 1, Nombre: Juan Pérez, Edad: 30
ID: 2, Nombre: Ana Gómez, Edad: 25
ID: 3, Nombre: Luis Rodríguez, Edad: 40
ID: 4, Nombre: María López, Edad: 28
ID: 5, Nombre: Carlos Sánchez, Edad: 35
